<div class="alert alert-block alert-danger">
    
<b>Python DB API</b>
    
</div>

#### PyODBC

In [1]:
# pip install pyodbc

In [2]:
import pyodbc

In [3]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 13 for SQL Server',
 'ODBC Driver 17 for SQL Server',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

<div class="alert alert-block alert-success">
    
<b>SQL Server Connection: SQL Server Authentication</b>
    
</div>

In [4]:
import pyodbc

def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER = server, DATABASE = database, UID = user, PWD = password)
        print("MS SQL Server Database connection successful")
    except Exception as err:
        print(f"Error:\n'{err}'")
    return conn

In [5]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' 
database = 'master' 
user = 'sa'               # system administrator
password = '---------'    # your password
conn = create_server_connection(driver, server, database, user, password)

MS SQL Server Database connection successful


In [6]:
conn.close()  # to close the sql server connection

<div class="alert alert-block alert-success">
    
<b>SQL Server Connection: Windows Authentication</b>
    
</div>

In [7]:
# connecting without password

In [8]:
# Method-1

conn = pyodbc.connect(
       "driver={SQL Server};"
       "server=DESKTOP-*****;"   # sql server, server_name
       "database=SampleSales;"
       "TRUSTED_CONNECTION=true;"
)


csr = conn.cursor()     # We use it to access the data in the database

conn.autocommit=True    # It allows the transactions made here to be reflected (commit) to the sql server, 
                        # if you do not commit, they will not be reflected.

In [9]:
conn.close()

In [10]:
# Method-2

conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"
conn = pyodbc.connect(conn_string)
csr = conn.cursor()

conn.autocommit=True
# conn.commit()  --  for desired transactions

<div class="alert alert-block alert-success">
    
<b>Create Database</b>
    
</div>

In [11]:
def create_database (conn, create_db_syntax):
    csr = conn.cursor()
    try:
        csr.execute(create_db_syntax)
        print("Database Created Succesfully")
    except Exception as err:
        print(f"Error:\n'{err}'")

In [12]:
create_db_syntax = 'CREATE DATABASE SAMPLEDB_1'

In [13]:
create_database(conn, create_db_syntax)

Database Created Succesfully


In [14]:
def execute_query(conn, query):
    csr = conn.cursor()
    try:
        csr.execute(query)
        # conn.commit()
        print("Query Successful!")
    except Exception as err:
        print(f"Error:\n'{err}'")

In [15]:
query = 'USE SAMPLEDB_1'

In [16]:
execute_query(conn, query)

Query Successful!


### ***Create Table***

In [17]:
query = 'CREATE TABLE TestA (\
           ID INT IDENTITY (1,1) NOT NULL, \
           FirstName VARCHAR(255) NOT NULL,\
           LastName VARCHAR(255) NOT NULL,\
           PRIMARY KEY(ID))'

In [18]:
execute_query(conn, query)

Query Successful!


### ***Insert Values***

In [19]:
csr.execute("INSERT TestA (FirstName, LastName) VALUES ('Bob', 'Marley')")

# conn.commit() -- if we didn't execute conn.autocommit=True

In [20]:
# parameterized

csr.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bruce', 'Lee')

In [21]:
csr.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Hanks')
csr.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bart', 'Simpson')

In [22]:
csr.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')
csr.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')
csr.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Owen', 'William')
csr.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

In [23]:
csr.execute("SELECT * FROM TestA").fetchall()

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (6, 'Jerry', 'Mouse'),
 (7, 'Owen', 'William'),
 (8, 'Stefan', 'Müller')]

### ***Updating and Deleting with rowcount***

In [24]:
csr.execute("DELETE FROM TestA WHERE FirstName = 'Jerry'")
print(csr.rowcount, 'row(s) deleted')

1 row(s) deleted


In [25]:
deleted = csr.execute("DELETE FROM TestA WHERE FirstName = ?", 'Stefan').rowcount
print(deleted, 'row(s) deleted')

1 row(s) deleted


In [26]:
csr.execute("SELECT * FROM TestA").fetchall()

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

In [27]:
csr.execute("UPDATE TestA SET FirstName='Natalie', LastName='Portman' WHERE ID=5")

print(csr.rowcount, 'row(s) updated')

1 row(s) updated


In [28]:
csr.execute("SELECT * FROM TestA").fetchall()

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Natalie', 'Portman'),
 (7, 'Owen', 'William')]

<div class="alert alert-block alert-success">
    
<b>Reading Data</b>
    
</div>

All SQL statements are executed using the Cursor execute() function. If the statement returns rows, such as a select statement, you can retrieve them using the Cursor fetch functions - fetchone(), fetchall(), fetchmany(). If there are no rows, fetchone() will return None, whereas fetchall() and fetchmany() will both return empty lists.

### ***fetchone***

In [29]:
csr.execute('SELECT FirstName, LastName FROM TestA')

In [30]:
row = csr.fetchone()  # it returns the first record of the table
row

('Bob', 'Marley')

In [31]:
row = csr.fetchone()  # it returns the next record of the table
row

('Bruce', 'Lee')

In [32]:
# The fetchone() function returns None when all rows have been retrieved.

In [33]:
row.FirstName

'Bruce'

In [34]:
row.LastName

'Lee'

In [35]:
csr.execute('select FirstName, LastName from TestA')
row = csr.fetchone()

print('LastName:', row[1])         # access by column index (zero-based)
print('LastName:', row.LastName)   # access by name

LastName: Marley
LastName: Marley


In [36]:
csr.execute('select FirstName, LastName from TestA')
while True:
    row = csr.fetchone()
    if not row:
        break
    print('FirstName:', row.FirstName)

FirstName: Bob
FirstName: Bruce
FirstName: Tom
FirstName: Bart
FirstName: Natalie
FirstName: Owen


In [37]:
csr.execute('select FirstName, LastName from TestA')
rows = csr.fetchall()

for row in rows:
    print(row.FirstName, row.LastName)

Bob Marley
Bruce Lee
Tom Hanks
Bart Simpson
Natalie Portman
Owen William


In [38]:
# parameters

csr.execute(""" select FirstName, LastName from TestA where FirstName = ? and LastName = ? """, ['Natalie', 'Portman'])
row = csr.fetchone()

if row:
    print(row)

('Natalie', 'Portman')


In [39]:
row = csr.execute("select count(*) as user_count from TestA").fetchone()

print('%s users' % row.user_count)

6 users


### ***fetchall***

In [40]:
csr.execute("SELECT * FROM TestA").fetchall()

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Natalie', 'Portman'),
 (7, 'Owen', 'William')]

In [41]:
csr.execute('SELECT ID, FirstName, LastName FROM TestA')
rows = csr.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Natalie', 'Portman'),
 (7, 'Owen', 'William')]

In [42]:
csr.execute("""SELECT ID, FirstName, LastName 
                FROM TestA""")
rows = csr.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Natalie', 'Portman'),
 (7, 'Owen', 'William')]

In [43]:
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

1 Bob Marley
2 Bruce Lee
3 Tom Hanks
4 Bart Simpson
5 Natalie Portman
7 Owen William


In [44]:
# parameterized

csr.execute("""SELECT ID, FirstName, LastName
               FROM TestA
               WHERE FirstName = ? and LastName = ?""", ['Owen', 'William'])
rows = csr.fetchall()
rows

[(7, 'Owen', 'William')]

### ***fetchmany***

In [45]:
csr.execute("""SELECT ID, FirstName, LastName
               FROM TestA""")
rows = csr.fetchmany(2)
rows

[(1, 'Bob', 'Marley'), (2, 'Bruce', 'Lee')]

In [46]:
csr.execute("""SELECT ID, FirstName, LastName
               FROM TestA""")
rows = csr.fetchmany(100)
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Natalie', 'Portman'),
 (7, 'Owen', 'William')]

### ***fetchval***

If you are selecting a single value you can use the fetchval convenience method.

If the statement generates a row, it returns the value of the first column of the first row. If there are no rows, None is returned

In [47]:
row_count = csr.execute("SELECT COUNT(*) FROM TestA").fetchone()[0]
row_count

6

In [48]:
row_count = csr.execute("SELECT COUNT(*) FROM TestA").fetchval()
row_count

6

In [49]:
csr.execute("SELECT MAX(ID) FROM TestA").fetchval()

7

In [50]:
csr.execute("SELECT FirstName FROM TestA").fetchval()

'Bob'

In [51]:
csr.execute("SELECT FirstName, LastName FROM TestA").fetchval()

'Bob'

In [52]:
csr.execute("SELECT * FROM TestA").fetchval()

1

### ***fetchone vs fetchval***

In [53]:
csr.execute("select coalesce(max(ID), 0) from TestA where FirstName = 'Eeee'").fetchone()[0]

0

In [54]:
csr.execute("select coalesce(count(*), 0) from TestA where FirstName = 'Eeee'").fetchone()[0]

0

fetchval is a better choice if the statement can return an empty set

In [55]:
# be careful!

csr.execute("""select FirstName from TestA where FirstName = 'Eeee'""").fetchone()[0]

TypeError: 'NoneType' object is not subscriptable

***The fetchone() call returns None. Python then attempts to apply [0] to the result (None[0]) which is not valid.***

In [56]:
# Preferred

csr.execute("""select FirstName from TestA where FirstName = 'Eeee'""").fetchval()

***The fetchval method was created just for this situation - it will detect the fact that there are no rows and will return None.***

***

<div class="alert alert-block alert-success">
    
<b>Connecting Different Databases</b>
    
</div>

In [57]:
query = 'USE SampleSales'

In [58]:
csr.execute(query)

In [59]:
csr.execute("SELECT * FROM product.brand").fetchall()

[(1, 'Electra'),
 (2, 'Haro'),
 (3, 'Redline'),
 (4, 'Cannondale'),
 (5, 'Schwinn'),
 (6, 'Giant'),
 (7, 'Sun Bicycles'),
 (8, 'Surly'),
 (9, 'Trek')]

In [61]:
csr.execute("SELECT * FROM product.category").fetchall()

[(1, 'Children Bicycles'),
 (2, 'Comfort Bicycles'),
 (3, 'Cruisers Bicycles'),
 (4, 'Cyclocross Bicycles'),
 (5, 'Electric Bikes'),
 (6, 'Mountain Bikes'),
 (7, 'Road Bikes')]

In [62]:
query =  """SELECT * FROM
 (
     SELECT brand_name, category_name, CAST(list_price AS INT) AS ListPrice 
     FROM product.product AS P, product.brand AS B, product.category AS C
     WHERE P.brand_id=B.brand_id AND P.category_id=C.category_id
 ) AS SubQ1
 PIVOT
 (
     AVG(ListPrice)
     FOR brand_name
     IN([Trek],[Schwinn],[Surly],[Redline],[Electra],[Cannondale],[Haro],[Sun Bicycles],[Giant])
 ) AS CategoryBrandPrices"""

In [63]:
csr.execute(query).fetchall()

[('Children Bicycles', 259, None, None, None, 329, None, 249, 109, 209),
 ('Comfort Bicycles', None, None, None, None, 771, None, None, 470, None),
 ('Cruisers Bicycles', None, 749, None, None, 785, None, None, 482, None),
 ('Cyclocross Bicycles', 3182, None, 1581, None, None, None, None, None, None),
 ('Electric Bikes', 3510, None, None, None, 2799, None, None, 1559, None),
 ('Mountain Bikes', 1925, None, 1302, 2172, None, None, 869, 832, None),
 ('Road Bikes', 3429, None, 1245, None, None, 442, None, None, None)]

In [64]:
query = """SELECT  
                RIGHT(email, LEN(email)-CHARINDEX('@', email)) AS DomainType,
                COUNT(RIGHT(email, LEN(email)-CHARINDEX('@', email))) AS NumofDomains
           FROM 
                sale.customer
           GROUP BY
                RIGHT(email, LEN(email)-CHARINDEX('@', email))
           ORDER BY
                NumofDomains DESC;"""

In [65]:
csr.execute(query).fetchall()

[('gmail.com', 305),
 ('yahoo.com', 305),
 ('msn.com', 291),
 ('aol.com', 275),
 ('hotmail.com', 269)]

In [66]:
conn.close()